In [ ]:
library(dplyr)

In [ ]:
participant_metadata = read.table("~/sc-online/notebook_data/pd/calico_donor_metadata.tsv", sep="\t", header=T)
participant_metadata$case_control = ifelse(
    participant_metadata$disease == "Control", "ctr", "pd"
)
head(participant_metadata)

In [ ]:
manifest_orig = read.table("~/sc-online/notebook_data/pd/Calico_Macosko_PD_GSA_Terra_Manifest_20240220.tsv", sep="\t", header=TRUE)
manifest_orig = manifest_orig[complete.cases(manifest_orig),]
manifest_orig$age = manifest_orig$Age 
manifest_orig$sex = manifest_orig$Sex
manifest_orig$case_control = ifelse(
    manifest_orig$Disease == "Control", "ctr", "pd"
)
manifest_orig$participant_id = manifest_orig$collaborator_participant_id
head(manifest_orig)


In [ ]:
repatient = read.table("~/sc-online/notebook_data/pd/calico_repatient_20240307.tsv", sep="\t", header=FALSE)
repatient$chip_well_barcode = repatient$V2
repatient$old_participant_id = repatient$V4
repatient$participant_id = repatient$V5
head(repatient)

In [ ]:
manifest = (
    manifest_orig
    %>% select(chip_well_barcode, participant_id, age, sex, case_control) 
    %>% left_join(repatient, by="chip_well_barcode", suffix=c("_orig", "_repatient"))
    %>% mutate(
        participant_id = ifelse(
            is.na(participant_id_repatient)
            , participant_id_orig
            , participant_id_repatient
        )
    )
)
unique(manifest$participant_id_repatient)

In [ ]:
manifest[manifest$participant_id %in% c("PDC091", "PDC110"),
    c("chip_well_barcode", "participant_id")
]

In [ ]:
head(manifest[,
    #manifest$participant_id %in% c("PDC091", "PDC110"),
    c("participant_id", "chip_well_barcode", "age", "sex", "case_control")])

In [ ]:
# write manifest to a tsv 
write.table(manifest[,c("participant_id", "chip_well_barcode", "age", "sex", "case_control")], "~/sc-online/notebook_data/pd/calico_donor_corrected_manifest_20240306.tsv", sep="\t", quote=FALSE, row.names=FALSE)